<a href="https://colab.research.google.com/github/lifei96/Gingko/blob/master/Model1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!git clone https://github.com/lifei96/Gingko.git

In [2]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import *
import csv
import random

Using TensorFlow backend.


In [0]:
f = open('Gingko/vectorized.csv')
data_0 = list()
data_1 = list()

In [0]:
csvreader = csv.reader(f)
for row in csvreader:
    row = [float(r) for r in row]
    if row[0] == 0:
        data_0.append(row)
    else:
        data_1.append(row)
f.close()

In [0]:
# balance the data
random.shuffle(data_0)
data_0 = data_0[:len(data_1)]

In [0]:
training_set = np.array(data_0[:700] + data_1[:700])
dev_set = np.array(data_0[700:] + data_1[700:])

training_labels = training_set[:, 0].astype(int)
training_data = training_set[:, 1:].astype('float32')
dev_labels = dev_set[:, 0].astype(int)
dev_data = dev_set[:, 1:].astype('float32')

In [0]:
# normalize features
_mean = np.mean(training_data, axis=0)
_std = np.std(training_data, axis=0)
training_data = (training_data - _mean) / _std
dev_data = (dev_data - _mean) / _std

In [8]:
model = Sequential()
model.add(Dense(units=50, activation='tanh', input_dim=6))
model.add(Dense(units=50, activation='tanh'))
model.add(Dense(units=2, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
model.fit(training_data, training_labels, epochs=100, batch_size=40)

In [10]:
model.evaluate(dev_data, dev_labels, batch_size=40)

336/336 [==============================] - 0s 117us/step


[0.44553375315098537, 0.8035714299905867]